In [1]:
#%%
from glob import glob
import pandas as pd
import numpy as np
from obspy import read_events


import obspy
from obspy import Catalog, UTCDateTime

import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
from matplotlib import cm


In [53]:
def convert_utc(yy,mm,dd,hh,mi,s):
    return UTCDateTime(int(yy), int(mm), int(dd), int(hh), int(mi), int(s))
    
eqcat = pd.read_csv("vn_cat.csv",index_col=0)
eqcat['utctime'] = eqcat.apply(lambda x: convert_utc(x.year, x.month, x.day, x.hour, x.minute, x.second),axis=1)
eqcat['eqid'] = eqcat.utctime.apply(lambda x: x.strftime("%Y%m%d%H%M%S"))

In [3]:
mlre = glob("MW_code/res/20*/*.out")
sdf = []
for i in range(0,len(mlre)):
    fml = mlre[i]
    eqid = fml.split("/")[-2]
    fmldata = open(fml,"r").readlines()
    mw = np.NaN
    try:
        mwinfo =  [a  for a in fmldata if "Mw (weighted):" in a][0]
        mw = round(float(mwinfo.split()[2]),1)
    except:
        pass
    
    sdict = {"eqid":eqid,"mw":mw}
    sdf.append(sdict)
#%%

sdf = pd.DataFrame(sdf)

In [54]:
eqcat = eqcat.merge(sdf,left_on="eqid",right_on="eqid",how = "outer")
eqcat.mw = eqcat.mw.fillna(eqcat.Ml)

In [57]:
cwbl = glob("ascii/*")


In [58]:
for fn in cwbl:
#     fn = cwbl[0]
    ev = fn.split("_")[-2]
    mw = round(float(eqcat.query("eqid == '{}'".format(ev)).reset_index(drop=True).mw[0]),1)
    with open(fn, "r") as f:
        contents = f.readlines()

    contents.insert(6, "%EventMw: {}\n".format(mw))
    nfn = "ascii_mw/" + fn.split("/")[-1]
    with open(nfn, "w") as f:
        contents = "".join(contents)
        f.write(contents)